In [1]:
import sys
import time
sys.path.append('../../')

source_dir = "../folder_example/source"
output_dir = "../folder_example/output"



In [2]:
from crimson.folder_sync.syncer import EditableFolderSyncer
from crimson.folder_sync.sync_handlers.editable_sync_handler import SyncFinal
from example_tools import ( 
    delete_dir,
    generate_dir,
    get_dir_structure,
    source_contents
)

## Sync Final

In [3]:
modified_content, modified_path = SyncFinal.sync_final("print('any content)", "root/path/filename.py")
print("modified_content: \n", modified_content) 
print()
print("modified_path: \n", modified_path) 

modified_content: 
 # modified content
print('any content)

modified_path: 
 root/path/filename_modified.py


In [4]:
import os
from typing import *

def sync_final(content: str, path: str) -> Tuple[str, str]:
    # 파일 내용 수정
    content += "\n# Copied One"
    
    # 파일 경로 수정
    directory, filename = os.path.split(path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_modified{ext}"
    new_path = os.path.join(directory, new_filename)
    
    return content, new_path

syncer = EditableFolderSyncer(
	source_dir=source_dir,
	output_dir=output_dir,
	sync_final=sync_final,
)

## Refresh Sync without clean_target

In [5]:
delete_dir(output_dir)
generate_dir(source_dir, source_contents)
get_dir_structure(source_dir)

example - INFO - Directory, '../folder_example/output',  deleted


{'file1.py': 'print("Hello world")\n',
 'file2.py': 'print("What a wonderful day!")\n',
 'lvl2/file3.py': 'print("File from lvl2.")\n'}

In [6]:
syncer.start()

FolderSyncer - INFO - Watching '../folder_example/source' for changes...


In [7]:
syncer.refresh_sync()

EditableFolderSyncer - INFO - Performing initial sync...
SyncHandler - INFO - Performing Refresh sync...
SyncHandler - INFO - Synced './../folder_example/source/file2.py' to './../folder_example/output/file2_modified.py'.
SyncHandler - INFO - Synced './../folder_example/source/file1.py' to './../folder_example/output/file1_modified.py'.
SyncHandler - INFO - Synced './../folder_example/source/lvl2/file3.py' to './../folder_example/output/lvl2/file3_modified.py'.
SyncHandler - INFO - Refresh sync completed.
EditableFolderSyncer - INFO - Initial sync completed.


In [8]:
previous_outdir_contents = {
	"./noise.py" : "print('I was written before you started sync.)"
}

delete_dir(output_dir)
generate_dir(output_dir, previous_outdir_contents)
get_dir_structure(output_dir)

example - INFO - Directory, '../folder_example/output',  deleted


{'noise.py': "print('I was written before you started sync.)"}

In [9]:
syncer.refresh_sync(clean_target=True)

time.sleep(0.5)



EditableFolderSyncer - INFO - Performing initial sync...
SyncHandler - INFO - Performing Refresh sync...
SyncHandler - INFO - Synced './../folder_example/source/file2.py' to './../folder_example/output/file2_modified.py'.
SyncHandler - INFO - Synced './../folder_example/source/file1.py' to './../folder_example/output/file1_modified.py'.
SyncHandler - INFO - Synced './../folder_example/source/lvl2/file3.py' to './../folder_example/output/lvl2/file3_modified.py'.
SyncHandler - INFO - Refresh sync completed.
EditableFolderSyncer - INFO - Initial sync completed.


In [10]:
source_dir_structure = get_dir_structure(source_dir)
source_dir_structure

{'file1.py': 'print("Hello world")\n',
 'file2.py': 'print("What a wonderful day!")\n',
 'lvl2/file3.py': 'print("File from lvl2.")\n'}

In [11]:
output_dir_structure = get_dir_structure(output_dir)
output_dir_structure

{'file1_modified.py': 'print("Hello world")\n\n# Copied One',
 'file2_modified.py': 'print("What a wonderful day!")\n\n# Copied One',
 'lvl2/file3_modified.py': 'print("File from lvl2.")\n\n# Copied One'}

In [12]:
previous_file_is_not_in_refreshed_output_dir = 'noise.py' not in output_dir_structure.keys()
assert previous_file_is_not_in_refreshed_output_dir
previous_file_is_not_in_refreshed_output_dir

True

In [13]:
original_content = open(f"{source_dir}/file1.py", "r").read()

open(f"{source_dir}/file1.py", "w").write(r'print("Modified!")')

"Syncer must sync the change."

'Syncer must sync the change.'

SyncHandler - INFO - Synced './../folder_example/source/file1.py' to './../folder_example/output/file1_modified.py'.


In [14]:
time.sleep(0.5)

new_source_content = open(f"{source_dir}/file1.py").read()
new_source_content

'print("Modified!")'

In [15]:
time.sleep(0.5)

output_path = f"{output_dir}/file1.py"

modified_content, modified_path = sync_final(new_source_content, output_path)

generated_content = open(modified_path).read()
generated_content

'print("Modified!")\n# Copied One'

In [16]:
time.sleep(0.5)

open(f"{source_dir}/file1.py", "w").write(original_content)

SyncHandler - INFO - Synced './../folder_example/source/file1.py' to './../folder_example/output/file1_modified.py'.


21

In [17]:
syncer.stop()

FolderSyncer - INFO - Stopped watching for changes.
